In [1]:
import numpy as np
import open3d as o3d
from scipy.optimize import least_squares
from sklearn.cluster import DBSCAN

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [19]:
pcd = o3d.io.read_point_cloud("filtered.pcd")
downpcd = pcd.voxel_down_sample(voxel_size=0.000001)
points = np.asarray(downpcd.points)
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
normals = np.asarray(downpcd.normals)

In [20]:
# Load your point cloud
point_cloud = o3d.io.read_point_cloud("filtered.pcd")
point_cloud.points = o3d.utility.Vector3dVector(points)  # Replace 'points' with your point cloud data

# Estimate normals for better visualization and context (optional but useful)
point_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Use Alpha Shape to create a concave hull of the point cloud (the outer boundary)
alpha = 0.0045  # Alpha value controls the tightness of the hull; adjust based on your data
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(point_cloud, alpha)

# Extract vertices (points) of the hull as the outer surface points
surface_points = np.asarray(mesh.vertices)

# Visualize the extracted surface points
surface_cloud = o3d.geometry.PointCloud()
surface_cloud.points = o3d.utility.Vector3dVector(surface_points)

# Optional: Estimate normals for the surface points for further processing or visualization
surface_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Visualize the surface points to ensure they correctly represent the outer shell
o3d.visualization.draw_geometries([surface_cloud], point_show_normal=True)

In [21]:
import numpy as np
from sklearn.decomposition import PCA

# Estimate normals with tuned parameters
surface_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Convert Open3D points to NumPy array for PCA
points_np = np.asarray(downpcd.points)

# Perform PCA to find the centroid and main components
pca = PCA(n_components=1)
pca.fit(points_np)
centroid = np.mean(points_np, axis=0)

# Convert normals to NumPy array
normals_np = np.asarray(surface_cloud.normals)

# Reorient normals to point outward from the centroid
for i in range(len(normals_np)):
    normal = normals_np[i]
    vector_to_point = points_np[i] - centroid
    # Flip the normal if it is pointing inward
    if np.dot(normal, vector_to_point) < 0:
        normals_np[i] = -normal

# Update the point cloud normals with the reoriented normals
surface_cloud.normals = o3d.utility.Vector3dVector(normals_np)
normals = np.asarray(surface_cloud.normals)
points = np.asarray(surface_cloud.points)

# Visualize the reoriented normals
o3d.visualization.draw_geometries([surface_cloud], point_show_normal=True)

In [22]:
import numpy as np
from sklearn.decomposition import PCA
import open3d as o3d

# Step 1: Estimate normals with tuned parameters
surface_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.2, max_nn=50))

# Step 2: Use Open3D's method for reorienting normals to be consistent
surface_cloud.orient_normals_consistent_tangent_plane(k=10)

# Step 3: Visualize the reoriented normals
o3d.visualization.draw_geometries([surface_cloud], point_show_normal=True)


In [12]:
import numpy as np
from scipy.optimize import least_squares

def calculate_cone_score(points, normals, apex, axis_direction, opening_angle, alignment_threshold=0.5):
    """
    Improved scoring function that considers distance, angular alignment, and density to evaluate cone fit.

    Parameters:
        points (ndarray): Nx3 array of 3D points on the cone.
        normals (ndarray): Nx3 array of normal vectors from the point cloud.
        apex (ndarray): 3-element array representing the cone's apex position.
        axis_direction (ndarray): 3-element array representing the cone's axis direction.
        opening_angle (float): Opening angle of the cone in radians.
        alignment_threshold (float): Threshold for normal alignment to consider a point valid.

    Returns:
        ndarray: Combined score values for each point considering surface fit, normal alignment, and density.
    """
    # Normalize the axis direction
    axis_direction = axis_direction / np.linalg.norm(axis_direction)

    # Compute vectors from apex to each point
    vectors_to_points = points - apex
    projection_lengths = np.dot(vectors_to_points, axis_direction)
    projections = projection_lengths[:, np.newaxis] * axis_direction

    # Calculate expected surface normals on the cone
    surface_normals = vectors_to_points - projections
    surface_normals /= np.linalg.norm(surface_normals, axis=1)[:, np.newaxis]

    # Adjust expected normals based on the cone’s geometry
    expected_normals = surface_normals + np.tan(opening_angle) * axis_direction
    expected_normals /= np.linalg.norm(expected_normals, axis=1)[:, np.newaxis]

    # Calculate dot products between the actual and expected normals
    normal_alignment = np.einsum('ij,ij->i', normals, expected_normals)
    angular_deviation = np.arccos(np.clip(normal_alignment, -1.0, 1.0))

    # Distance-based score: difference between actual and expected distances
    perpendicular_distance = np.linalg.norm(vectors_to_points - projections, axis=1)
    expected_distance = np.tan(opening_angle) * projection_lengths
    distance_score = np.exp(-10 * np.abs(perpendicular_distance - expected_distance))

    # Angular alignment score: penalizes points with high angular deviation
    angular_score = np.exp(-10 * angular_deviation)

    # Density-based score: reward points that follow expected density distribution
    density_score = compute_density_score(points, apex, axis_direction)

    # Combined score considering distance, angular, and density scores
    combined_score = (distance_score * angular_score * density_score) * (angular_deviation < alignment_threshold)

    return combined_score

def compute_density_score(points, apex, axis_direction):
    """
    Computes a density score based on the expected density distribution along the cone's height.

    Parameters:
        points (ndarray): Nx3 array of 3D points on the cone.
        apex (ndarray): 3-element array representing the cone's apex position.
        axis_direction (ndarray): 3-element array representing the cone's axis direction.

    Returns:
        ndarray: Density score values for each point.
    """
    # Calculate distances from apex along the cone axis
    distances_to_apex = np.dot(points - apex, axis_direction)
    distances_to_apex = np.clip(distances_to_apex, 0, None)  # Clip negative values (below apex)

    # Normalize distances to get a score that increases with proximity to the base
    max_distance = distances_to_apex.max() if distances_to_apex.max() > 0 else 1
    density_score = 1 - (distances_to_apex / max_distance)  # Higher score closer to the base

    return density_score

def residuals(params, points):
    x0, y0, z0, a, b, c, theta, height = params
    direction_vector = np.array([a, b, c])
    direction_vector /= np.linalg.norm(direction_vector)  

    vector_to_point = points - np.array([x0, y0, z0])
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector

    perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
    expected_distance = np.tan(theta) * projection_length

    residual = perpendicular_distance - expected_distance

    below_apex = projection_length < 0
    above_base = projection_length > height
    residual[below_apex | above_base] = np.inf  

    residual = np.nan_to_num(residual, nan=1e6, posinf=1e6, neginf=1e6)

    return residual

def fit_cone(points, min_height=0.0, max_height=1.0):
    apex = points[0]  
    base_points = points[1:5]  
    base_center = np.mean(base_points, axis=0)  
    height_vector = base_center - apex  
    height = np.linalg.norm(height_vector)
    initial_direction = height_vector / height  
    initial_angle = np.arctan(np.mean(np.linalg.norm(base_points - base_center, axis=1)) / height)  

    if height < min_height or height > max_height:
        return None  

    initial_guess = [*apex, *initial_direction, initial_angle, height]

    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x  

def ransac_cone(points, normals, threshold, iterations, alignment_threshold=0.5, min_height=0.0, max_height=0.04):
    best_inliers = []
    best_params = None
    best_score = 0

    for i in range(iterations):
        print(f"Iteration: {i + 1}")  
        sample_indices = np.random.choice(points.shape[0], 5, replace=False)
        sample = points[sample_indices]
        apex = sample[0]  
        base_points = sample[1:]  

        params = fit_cone(np.vstack([apex, base_points]), min_height=min_height, max_height=max_height)
        if params is None:
            print(f"Skipped due to height constraint.")  
            continue  

        x0, y0, z0, a, b, c, theta, height = params
        print(f"Height of the fitted cone: {height}")  
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  

        inliers = []
        for idx, point in enumerate(points):
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            distance = np.abs(perpendicular_distance - expected_distance)

            if distance < threshold and 0 <= projection_length <= height:
                inliers.append((point, idx))

        if len(inliers) == 0:
            print("No inliers found, skipping iteration.")
            continue

        inlier_points = np.array([pt for pt, _ in inliers])
        inlier_normals = normals[[idx for _, idx in inliers]]
        inlier_scores = calculate_cone_score(inlier_points, inlier_normals, np.array([x0, y0, z0]), direction_vector, theta, alignment_threshold)

        refined_inliers = [(inliers[idx][0], inlier_scores[idx]) for idx in range(len(inliers)) if inlier_scores[idx] > 0]

        if len(refined_inliers) == 0:
            print("No refined inliers after scoring, skipping iteration.")
            continue

        total_score = sum([score for _, score in refined_inliers])

        if total_score > best_score:
            best_inliers = refined_inliers
            best_params = params
            best_score = total_score

    print(f"Best Score: {best_score}, Inliers Count: {len(best_inliers)}")
    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)

In [24]:
import numpy as np
from scipy.optimize import least_squares

def fit_cone(points):
    def residuals(params, points):
        # Unpack cone parameters: apex (x0, y0, z0), direction (a, b, c), and opening angle (theta)
        x0, y0, z0, a, b, c, theta, height = params
        apex = np.array([x0, y0, z0])
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector

        # Calculate residuals
        vector_to_point = points - apex
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(theta) * projection_length

        # Penalize points that are outside the finite height of the cone
        below_apex = projection_length < 0
        above_base = projection_length > height
        residual = perpendicular_distance - expected_distance
        residual[below_apex | above_base] = np.inf  # Large penalty for points outside the height range

        # Replace NaN or Inf values with a large finite penalty
        residual = np.nan_to_num(residual, nan=1e6, posinf=1e6, neginf=1e6)

        return residual

    # Initial guesses based on sampled points
    apex = points[0]  # First point is the apex
    base_points = points[1:5]  # Next four points define the base
    base_center = np.mean(base_points, axis=0)  # Approximate the base circle's center
    height_vector = base_center - apex  # Vector from apex to base center
    height = np.linalg.norm(height_vector)
    initial_direction = height_vector / height  # Normalize the direction vector
    initial_angle = np.arctan(np.mean(np.linalg.norm(base_points - base_center, axis=1)) / height)  # Estimate opening angle

    # Initial guess for parameters: apex (x0, y0, z0), direction (a, b, c), opening angle (theta), and height
    initial_guess = [*apex, *initial_direction, initial_angle, height]

    # Optimize parameters to fit the cone model
    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x  # Return optimized parameters

def ransac_cone(points, threshold, iterations):
    best_inliers = []
    best_params = None

    for _ in range(iterations):
        # Sample 5 points: 1 for the apex and 4 for defining the base
        sample_indices = np.random.choice(points.shape[0], 5, replace=False)
        sample = points[sample_indices]
        apex = sample[0]  # First sampled point is the apex
        base_points = sample[1:]  # Remaining 4 points define the base

        # Fit cone using the sampled apex and base points
        params = fit_cone(np.vstack([apex, base_points]))
        x0, y0, z0, a, b, c, theta, height = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize direction vector

        # Evaluate inliers
        inliers = []
        for point in points:
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            distance = np.abs(perpendicular_distance - expected_distance)

            # Check if the point is an inlier and within the finite height of the cone
            if distance < threshold and 0 <= projection_length <= height:
                inliers.append(point)

        # Update best fit if current fit has more inliers
        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_params = params

    return best_params, np.array(best_inliers)

In [37]:
def calculate_cone_score(points, normals, apex, axis_direction, opening_angle, alignment_threshold=0.5):
    """
    Calculates a score for each point based on how consistent its angle is relative to the cone's opening angle,
    how close it is to the cone surface, and how aligned its normal is to the expected cone normals.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        normals (ndarray): Nx3 array of normal vectors from the point cloud (optional, can be None).
        apex (ndarray): 3-element array representing the cone's apex.
        axis_direction (ndarray): 3-element array representing the cone's axis direction.
        opening_angle (float): Opening angle of the cone in radians.
        alignment_threshold (float): Threshold for normal alignment to consider a point valid.

    Returns:
        ndarray: Score values for each point.
    """
    # Normalize the axis direction
    axis_direction = axis_direction / np.linalg.norm(axis_direction)

    # Compute vectors from apex to each point
    vectors_to_points = points - apex
    projection_lengths = np.dot(vectors_to_points, axis_direction)
    projections = projection_lengths[:, np.newaxis] * axis_direction

    # Calculate the angle between the cone's axis and each point's position vector
    angles = np.arccos(np.clip(np.dot(vectors_to_points, axis_direction) / np.linalg.norm(vectors_to_points, axis=1), -1, 1))

    # Calculate the angular consistency score based on deviation from the opening angle
    angular_consistency = np.exp(-10 * np.abs(angles - opening_angle))  # Penalize deviation from opening angle

    # Calculate expected distance from the cone surface (perpendicular distance to the axis)
    perpendicular_distance = np.linalg.norm(vectors_to_points - projections, axis=1)
    expected_distance = np.tan(opening_angle) * projection_lengths

    # Distance-based score: exponential penalty for points far from the expected cone surface
    distance_score = np.exp(-10 * np.abs(perpendicular_distance - expected_distance))

    # If normals are provided, include normal alignment in the score
    if normals is not None:
        # Calculate expected surface normals (cone normals) based on geometry
        surface_normals = vectors_to_points - projections
        surface_normals /= np.linalg.norm(surface_normals, axis=1)[:, np.newaxis]

        # Compare point cloud normals to expected cone normals
        normal_alignment = np.abs(np.einsum('ij,ij->i', normals, surface_normals))
        angular_score = np.exp(-50 * (normal_alignment - 1) ** 2)  # Sharply reward good alignment

        # Combine distance score, angular consistency, and normal alignment
        total_score = distance_score * angular_consistency * angular_score
    else:
        # Combine distance score and angular consistency without normals
        total_score = distance_score * angular_consistency

    return total_score

def ransac_cone(points, normals, threshold, iterations, alignment_threshold=0.5):
    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        # Sample 5 points: 1 for the apex and 4 for defining the base
        sample_indices = np.random.choice(points.shape[0], 5, replace=False)
        sample = points[sample_indices]
        apex = sample[0]  
        base_points = sample[1:]  

        # Fit cone using the sampled apex and base points
        params = fit_cone(np.vstack([apex, base_points]))
        x0, y0, z0, a, b, c, theta, height = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  

        # Evaluate inliers and calculate scores
        inliers = []
        for point in points:
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            distance = np.abs(perpendicular_distance - expected_distance)

            # Check if the point is an inlier and within the finite height of the cone
            if distance < threshold and 0 <= projection_length <= height:
                inliers.append(point)

        inlier_points = np.array(inliers)

        # Calculate scores for the inliers
        inlier_normals = normals if normals is not None else None
        scores = calculate_cone_score(inlier_points, inlier_normals, np.array([x0, y0, z0]), direction_vector, theta, alignment_threshold)

        # Calculate total score for this iteration
        total_score = np.sum(scores)

        # Update best fit if current fit has a higher score
        if total_score > best_score:
            best_inliers = inliers
            best_params = params
            best_score = total_score

    return best_params, np.array(best_inliers)

In [42]:
def ransac_cone(points, normals, threshold, iterations, alignment_threshold=0.5):
    best_inliers = []
    best_params = None
    best_score = 0

    for i in range(iterations):
        print(f"Iteration: {i + 1}")  # Track the iteration number
        # Sample 5 points: 1 for the apex and 4 for defining the base
        sample_indices = np.random.choice(points.shape[0], 5, replace=False)
        sample = points[sample_indices]
        apex = sample[0]  
        base_points = sample[1:]  

        # Fit cone using the sampled apex and base points
        params = fit_cone(np.vstack([apex, base_points]))
        if params is None:
            print("Skipped due to failure in fitting cone.")
            continue

        x0, y0, z0, a, b, c, theta, height = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize direction vector

        # Evaluate inliers
        inliers = []
        inlier_indices = []  # Track indices of inliers for normals extraction
        for idx, point in enumerate(points):
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            distance = np.abs(perpendicular_distance - expected_distance)

            # Check if the point is an inlier and within the finite height of the cone
            if distance < threshold and 0 <= projection_length <= height:
                inliers.append(point)
                inlier_indices.append(idx)  # Save index of inlier

        inlier_points = np.array(inliers)

        if len(inlier_points) == 0:
            print("No inliers found, skipping iteration.")
            continue  # Skip this iteration if no inliers

        # Extract the corresponding normals for inlier points
        inlier_normals = normals[inlier_indices] if normals is not None else None

        # Calculate scores for the inliers
        scores = calculate_cone_score(inlier_points, inlier_normals, np.array([x0, y0, z0]), direction_vector, theta, alignment_threshold)

        if scores.size == 0:
            print("No scores calculated, skipping iteration.")
            continue

        # Calculate total score for this iteration
        total_score = np.sum(scores)

        # Update best fit if current fit has a higher score
        if total_score > best_score:
            best_inliers = inliers
            best_params = params
            best_score = total_score

    print(f"Best Score: {best_score}, Inliers Count: {len(best_inliers)}")
    return best_params, np.array(best_inliers)

In [64]:
def calculate_cone_score(points, normals, apex, axis_direction, opening_angle, alignment_threshold=0.5):
    """
    Calculates a score for each point based on angular consistency, distance from the cone surface, and normal alignment.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        normals (ndarray): Nx3 array of normal vectors.
        apex (ndarray): 3-element array representing the cone's apex.
        axis_direction (ndarray): 3-element array representing the cone's axis direction.
        opening_angle (float): Opening angle of the cone in radians.
        alignment_threshold (float): Threshold for normal alignment to consider a point valid.

    Returns:
        ndarray: Score values for each point.
    """
    # Normalize the axis direction
    axis_direction = axis_direction / np.linalg.norm(axis_direction)

    # Compute vectors from apex to each point
    vectors_to_points = points - apex
    projection_lengths = np.dot(vectors_to_points, axis_direction)
    projections = projection_lengths[:, np.newaxis] * axis_direction

    # Calculate the angle between the cone's axis and each point's position vector
    angles = np.arccos(np.clip(np.dot(vectors_to_points, axis_direction) / np.linalg.norm(vectors_to_points, axis=1), -1, 1))

    # Strictly penalize points that deviate too much from the expected opening angle
    angle_deviation_threshold = np.deg2rad(5)  # Allow up to 5 degrees of angular deviation
    angular_consistency = np.where(np.abs(angles - opening_angle) < angle_deviation_threshold, 
                                   np.exp(-50 * np.abs(angles - opening_angle)), 0)  # Angular penalty

    # Calculate expected distance from the cone surface (perpendicular distance to the axis)
    perpendicular_distance = np.linalg.norm(vectors_to_points - projections, axis=1)
    expected_distance = np.tan(opening_angle) * projection_lengths

    # Distance-based score: penalize points that deviate from the cone surface
    distance_score = np.exp(-20 * np.abs(perpendicular_distance - expected_distance))

    # If normals are provided, include normal alignment in the score
    if normals is not None:
        # Calculate expected surface normals (cone normals) based on geometry
        surface_normals = vectors_to_points - projections
        surface_normals /= np.linalg.norm(surface_normals, axis=1)[:, np.newaxis]

        # Compare point cloud normals to expected cone normals
        normal_alignment = np.abs(np.einsum('ij,ij->i', normals, surface_normals))
        angular_score = np.exp(-50 * (normal_alignment - 1) ** 2)  # Sharply reward good alignment

        # Combine distance score, angular consistency, and normal alignment
        total_score = distance_score * angular_consistency * angular_score
    else:
        # Combine distance score and angular consistency without normals
        total_score = distance_score * angular_consistency

    return total_score

def ransac_cone(points, normals, threshold, iterations, alignment_threshold=0.5):
    best_inliers = []
    best_params = None
    best_score = 0

    for i in range(iterations):
        # Sample 5 points: 1 for the apex and 4 for defining the base
        sample_indices = np.random.choice(points.shape[0], 5, replace=False)
        sample = points[sample_indices]
        apex = sample[0]  
        base_points = sample[1:]  

        # Fit cone using the sampled apex and base points
        params = fit_cone(np.vstack([apex, base_points]))
        if params is None:
            continue

        x0, y0, z0, a, b, c, theta, height = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize direction vector

        # Evaluate inliers
        inliers = []
        for idx, point in enumerate(points):
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            distance = np.abs(perpendicular_distance - expected_distance)

            # Check if the point is an inlier, within the finite height, and within angle deviation
            if distance < threshold and 0 <= projection_length <= height:
                inliers.append(point)

        inlier_points = np.array(inliers)
        inlier_normals = normals if normals is not None else None

        # Calculate scores for the inliers
        scores = calculate_cone_score(inlier_points, inlier_normals, np.array([x0, y0, z0]), direction_vector, theta, alignment_threshold)

        # Calculate total score for this iteration
        total_score = np.sum(scores)

        # Update best fit if current fit has a higher score
        if total_score > best_score:
            best_inliers = inliers
            best_params = params
            best_score = total_score

    return best_params, np.array(best_inliers)

In [66]:
def ransac_cone(points, normals, threshold, iterations, alignment_threshold=0.5):
    best_inliers = []
    best_params = None
    best_score = 0

    for i in range(iterations):
        # Sample 5 points: 1 for the apex and 4 for defining the base
        sample_indices = np.random.choice(points.shape[0], 5, replace=False)
        sample = points[sample_indices]
        apex = sample[0]  
        base_points = sample[1:]  

        # Fit cone using the sampled apex and base points
        params = fit_cone(np.vstack([apex, base_points]))
        if params is None:
            continue

        x0, y0, z0, a, b, c, theta, height = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize direction vector

        # Evaluate inliers
        inliers = []
        inlier_indices = []  # Track indices of inliers for normals extraction
        for idx, point in enumerate(points):
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            distance = np.abs(perpendicular_distance - expected_distance)

            # Check if the point is an inlier and within the finite height of the cone
            if distance < threshold and 0 <= projection_length <= height:
                inliers.append(point)
                inlier_indices.append(idx)  # Save index of inlier

        inlier_points = np.array(inliers)

        if len(inlier_points) == 0:
            print("No inliers found, skipping iteration.")
            continue  # Skip this iteration if no inliers

        # Extract the corresponding normals for inlier points
        inlier_normals = normals[inlier_indices] if normals is not None else None

        # Calculate scores for the inliers
        scores = calculate_cone_score(inlier_points, inlier_normals, np.array([x0, y0, z0]), direction_vector, theta, alignment_threshold)

        if scores.size == 0:
            print("No scores calculated, skipping iteration.")
            continue

        # Calculate total score for this iteration
        total_score = np.sum(scores)

        # Update best fit if current fit has a higher score
        if total_score > best_score:
            best_inliers = inliers
            best_params = params
            best_score = total_score

    print(f"Best Score: {best_score}, Inliers Count: {len(best_inliers)}")
    return best_params, np.array(best_inliers)

In [76]:
import numpy as np
from scipy.optimize import least_squares

def fit_cone(points):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta, height = params
        apex = np.array([x0, y0, z0])
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector

        vector_to_point = points - apex
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(theta) * projection_length

        # Penalize points outside the height range of the cone
        below_apex = projection_length < 0
        above_base = projection_length > height
        residual = perpendicular_distance - expected_distance
        residual[below_apex | above_base] = np.inf

        residual = np.nan_to_num(residual, nan=1e6, posinf=1e6, neginf=1e6)
        return residual

    apex = points[0]  # First point is the apex
    base_points = points[1:5]  # Next four points define the base
    base_center = np.mean(base_points, axis=0)
    height_vector = base_center - apex
    height = np.linalg.norm(height_vector)
    initial_direction = height_vector / height
    initial_angle = np.arctan(np.mean(np.linalg.norm(base_points - base_center, axis=1)) / height)

    initial_guess = [*apex, *initial_direction, initial_angle, height]
    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def ransac_cone(points, normals, threshold, iterations, alignment_threshold=0.5):
    best_inliers = []
    best_params = None
    best_score = 0

    # Split points into top (for apex sampling) and base (for base point sampling)
    split_height = np.percentile(points[:, 2], 90)  # Top 10% of points are considered potential apex points
    top_points = points[points[:, 2] > split_height]  # Points near the apex (top 10%)
    base_points = points[points[:, 2] <= split_height]  # Points near the base (bottom 90%)

    for i in range(iterations):
        # Sample 1 point from the top points (likely apex) and 4 from the base points
        apex_sample = top_points[np.random.choice(top_points.shape[0], 1, replace=False)]
        base_sample = base_points[np.random.choice(base_points.shape[0], 4, replace=False)]

        sample = np.vstack([apex_sample, base_sample])  # Combine the apex and base samples

        # Fit cone using the sampled apex and base points
        params = fit_cone(sample)
        if params is None:
            continue

        x0, y0, z0, a, b, c, theta, height = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)

        inliers = []
        inlier_indices = []
        for idx, point in enumerate(points):
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            distance = np.abs(perpendicular_distance - expected_distance)

            # Check if the point is an inlier and within the finite height of the cone
            if distance < threshold and 0 <= projection_length <= height:
                inliers.append(point)
                inlier_indices.append(idx)

        inlier_points = np.array(inliers)
        if len(inlier_points) == 0:
            continue

        inlier_normals = normals[inlier_indices] if normals is not None else None
        scores = calculate_cone_score(inlier_points, inlier_normals, np.array([x0, y0, z0]), direction_vector, theta, alignment_threshold)

        if scores.size == 0:
            continue

        total_score = np.sum(scores)
        if total_score > best_score:
            best_inliers = inliers
            best_params = params
            best_score = total_score

    print(f"Best Score: {best_score}, Inliers Count: {len(best_inliers)}")
    return best_params, np.array(best_inliers)

def calculate_cone_score(points, normals, apex, axis_direction, opening_angle, alignment_threshold=0.5):
    axis_direction /= np.linalg.norm(axis_direction)
    vectors_to_points = points - apex
    projection_lengths = np.dot(vectors_to_points, axis_direction)
    projections = projection_lengths[:, np.newaxis] * axis_direction
    angles = np.arccos(np.clip(np.dot(vectors_to_points, axis_direction) / np.linalg.norm(vectors_to_points, axis=1), -1, 1))

    angle_deviation_threshold = np.deg2rad(5)
    angular_consistency = np.where(np.abs(angles - opening_angle) < angle_deviation_threshold, 
                                   np.exp(-100 * np.abs(angles - opening_angle)), 0)

    perpendicular_distance = np.linalg.norm(vectors_to_points - projections, axis=1)
    expected_distance = np.tan(opening_angle) * projection_lengths
    radius_change = np.abs(perpendicular_distance - expected_distance)

    distance_score = np.exp(-100 * radius_change)

    if normals is not None:
        surface_normals = vectors_to_points - projections
        surface_normals /= np.linalg.norm(surface_normals, axis=1)[:, np.newaxis]
        normal_alignment = np.abs(np.einsum('ij,ij->i', normals, surface_normals))
        angular_score = np.exp(-50 * (normal_alignment - 1) ** 2)

        total_score = distance_score * angular_consistency * angular_score
    else:
        total_score = distance_score * angular_consistency

    return total_score


In [79]:
import numpy as np
from scipy.optimize import least_squares

def fit_cone(points):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta, height = params
        apex = np.array([x0, y0, z0])
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize direction vector

        vector_to_point = points - apex
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(theta) * projection_length

        # Penalize points that are outside the finite height of the cone
        below_apex = projection_length < 0
        above_base = projection_length > height
        residual = perpendicular_distance - expected_distance
        residual[below_apex | above_base] = np.inf  # Large penalty for points outside the height range

        residual = np.nan_to_num(residual, nan=1e6, posinf=1e6, neginf=1e6)
        return residual

    apex = points[0]  # First point is the apex
    base_points = points[1:5]  # Next four points define the base
    base_center = np.mean(base_points, axis=0)
    height_vector = base_center - apex
    height = np.linalg.norm(height_vector)
    initial_direction = height_vector / height
    initial_angle = np.arctan(np.mean(np.linalg.norm(base_points - base_center, axis=1)) / height)

    initial_guess = [*apex, *initial_direction, initial_angle, height]
    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def calculate_cone_score(points, normals, apex, axis_direction, opening_angle, alignment_threshold=0.5):
    axis_direction /= np.linalg.norm(axis_direction)

    vectors_to_points = points - apex
    projection_lengths = np.dot(vectors_to_points, axis_direction)
    projections = projection_lengths[:, np.newaxis] * axis_direction
    angles = np.arccos(np.clip(np.dot(vectors_to_points, axis_direction) / np.linalg.norm(vectors_to_points, axis=1), -1, 1))

    angle_deviation_threshold = np.deg2rad(5)
    angular_consistency = np.where(np.abs(angles - opening_angle) < angle_deviation_threshold, 
                                   np.exp(-100 * np.abs(angles - opening_angle)), 0)

    perpendicular_distance = np.linalg.norm(vectors_to_points - projections, axis=1)
    expected_distance = np.tan(opening_angle) * projection_lengths
    radius_change = np.abs(perpendicular_distance - expected_distance)

    # Penalize points with nearly constant radius (cylinder-like behavior)
    radius_growth = np.gradient(perpendicular_distance)
    constant_radius_penalty = np.exp(-100 * np.abs(radius_growth - expected_distance / projection_lengths))
    distance_score = np.exp(-100 * radius_change) * constant_radius_penalty

    if normals is not None:
        surface_normals = vectors_to_points - projections
        surface_normals /= np.linalg.norm(surface_normals, axis=1)[:, np.newaxis]
        normal_alignment = np.abs(np.einsum('ij,ij->i', normals, surface_normals))
        angular_score = np.exp(-50 * (normal_alignment - 1) ** 2)

        total_score = distance_score * angular_consistency * angular_score
    else:
        total_score = distance_score * angular_consistency

    return total_score

def ransac_cone(points, normals, threshold, iterations, alignment_threshold=0.5):
    best_inliers = []
    best_params = None
    best_score = 0

    split_height = np.percentile(points[:, 2], 90)  # Top 10% for apex sampling
    top_points = points[points[:, 2] > split_height]  # Points near the apex
    base_points = points[points[:, 2] <= split_height]  # Points near the base

    for i in range(iterations):
        # Sample 1 apex point and 4 base points
        apex_sample = top_points[np.random.choice(top_points.shape[0], 1, replace=False)]
        base_sample = base_points[np.random.choice(base_points.shape[0], 4, replace=False)]

        sample = np.vstack([apex_sample, base_sample])

        # Fit cone using the sampled apex and base points
        params = fit_cone(sample)
        if params is None:
            continue

        x0, y0, z0, a, b, c, theta, height = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)

        inliers = []
        inlier_indices = []
        for idx, point in enumerate(points):
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            distance = np.abs(perpendicular_distance - expected_distance)

            # Check if the point is an inlier and within the finite height of the cone
            if distance < threshold and 0 <= projection_length <= height:
                inliers.append(point)
                inlier_indices.append(idx)

        inlier_points = np.array(inliers)
        if len(inlier_points) == 0:
            continue

        inlier_normals = normals[inlier_indices] if normals is not None else None
        scores = calculate_cone_score(inlier_points, inlier_normals, np.array([x0, y0, z0]), direction_vector, theta, alignment_threshold)

        if scores.size == 0:
            continue

        total_score = np.sum(scores)
        if total_score > best_score:
            best_inliers = inliers
            best_params = params
            best_score = total_score

    print(f"Best Score: {best_score}, Inliers Count: {len(best_inliers)}")
    return best_params, np.array(best_inliers)

In [85]:
import numpy as np
from scipy.optimize import least_squares

def fit_cone(points):
    def residuals(params, points):
        # Unpack cone parameters: apex (x0, y0, z0), direction (a, b, c), opening angle (theta), and height
        x0, y0, z0, a, b, c, theta, height = params
        apex = np.array([x0, y0, z0])
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector

        vector_to_point = points - apex
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(theta) * projection_length

        # Penalize points outside the height range of the cone
        below_apex = projection_length < 0
        above_base = projection_length > height
        residual = perpendicular_distance - expected_distance
        residual[below_apex | above_base] = np.inf  # Large penalty for points outside the height range

        residual = np.nan_to_num(residual, nan=1e6, posinf=1e6, neginf=1e6)
        return residual

    apex = points[0]  # First point is the apex
    base_points = points[1:5]  # Next four points define the base
    base_center = np.mean(base_points, axis=0)
    height_vector = base_center - apex
    height = np.linalg.norm(height_vector)
    initial_direction = height_vector / height
    initial_angle = np.arctan(np.mean(np.linalg.norm(base_points - base_center, axis=1)) / height)

    initial_guess = [*apex, *initial_direction, initial_angle, height]
    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def calculate_cone_score(points, apex, axis_direction, opening_angle):
    axis_direction /= np.linalg.norm(axis_direction)

    # Compute vectors from apex to each point
    vectors_to_points = points - apex
    projection_lengths = np.dot(vectors_to_points, axis_direction)
    projections = projection_lengths[:, np.newaxis] * axis_direction
    perpendicular_distance = np.linalg.norm(vectors_to_points - projections, axis=1)
    expected_distance = np.tan(opening_angle) * projection_lengths

    # Ensure the radius increases as you move away from the apex (for cones)
    radius_increasing = np.diff(perpendicular_distance) >= 0
    decreasing_penalty = np.where(np.hstack(([True], radius_increasing)), 0, np.inf)

    # Calculate score based on expected radius growth (with a sharp penalty for radius decrease)
    radius_growth_penalty = np.exp(-100 * np.abs(perpendicular_distance - expected_distance)) + decreasing_penalty

    return radius_growth_penalty

def ransac_cone(points, threshold, iterations, min_height=None, max_height=None):
    best_inliers = []
    best_params = None
    best_score = 0

    # Divide points into top (for apex) and base (for base points)
    split_height = np.percentile(points[:, 2], 90)  # Top 10% for apex sampling
    top_points = points[points[:, 2] > split_height]
    base_points = points[points[:, 2] <= split_height]

    for i in range(iterations):
        # Sample apex from top points and base points from the bottom 90%
        apex_sample = top_points[np.random.choice(top_points.shape[0], 1, replace=False)]
        base_sample = base_points[np.random.choice(base_points.shape[0], 4, replace=False)]

        sample = np.vstack([apex_sample, base_sample])

        # Fit cone using the sampled apex and base points
        params = fit_cone(sample)
        if params is None:
            continue

        x0, y0, z0, a, b, c, theta, height = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)

        inliers = []
        inlier_indices = []
        for idx, point in enumerate(points):
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            distance = np.abs(perpendicular_distance - expected_distance)

            # Check if the point is an inlier and within the finite height of the cone
            if distance < threshold and 0 <= projection_length <= height:
                inliers.append(point)
                inlier_indices.append(idx)

        inlier_points = np.array(inliers)
        if len(inlier_points) == 0:
            continue

        # Calculate score for the current fit based on cone properties (radius growth and angle)
        scores = calculate_cone_score(inlier_points, np.array([x0, y0, z0]), direction_vector, theta)

        total_score = np.sum(scores)
        if total_score > best_score:
            best_inliers = inliers
            best_params = params
            best_score = total_score

    print(f"Best Score: {best_score}, Inliers Count: {len(best_inliers)}")
    return best_params, np.array(best_inliers)

In [91]:
import numpy as np
from scipy.optimize import least_squares

def fit_cone(points):
    def residuals(params, points):
        # Unpack cone parameters: apex (x0, y0, z0), direction (a, b, c), opening angle (theta), and height
        x0, y0, z0, a, b, c, theta, height = params
        apex = np.array([x0, y0, z0])
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector

        vector_to_point = points - apex
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(theta) * projection_length

        # Penalize points outside the height range of the cone
        below_apex = projection_length < 0
        above_base = projection_length > height
        residual = perpendicular_distance - expected_distance
        residual[below_apex | above_base] = np.inf  # Large penalty for points outside the height range

        residual = np.nan_to_num(residual, nan=1e6, posinf=1e6, neginf=1e6)
        return residual

    apex = points[0]  # First point is the apex
    base_points = points[1:5]  # Next four points define the base
    base_center = np.mean(base_points, axis=0)
    height_vector = base_center - apex
    height = np.linalg.norm(height_vector)
    initial_direction = height_vector / height
    initial_angle = np.arctan(np.mean(np.linalg.norm(base_points - base_center, axis=1)) / height)

    initial_guess = [*apex, *initial_direction, initial_angle, height]
    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def calculate_cone_score(points, normals, apex, axis_direction, opening_angle, epsilon, alpha):
    axis_direction /= np.linalg.norm(axis_direction)

    # Compute vectors from apex to each point
    vectors_to_points = points - apex
    projection_lengths = np.dot(vectors_to_points, axis_direction)
    projections = projection_lengths[:, np.newaxis] * axis_direction
    perpendicular_distance = np.linalg.norm(vectors_to_points - projections, axis=1)
    expected_distance = np.tan(opening_angle) * projection_lengths

    # Distance Condition: check if the point is within the epsilon-radius band
    distance_compatible = np.abs(perpendicular_distance - expected_distance) <= epsilon

    # Compute expected surface normals (unit vectors pointing outward from the cone's surface)
    surface_normals = vectors_to_points - projections
    surface_normals /= np.linalg.norm(surface_normals, axis=1)[:, np.newaxis]

    # Normal Condition: check if the normals deviate within the alpha angle threshold
    cos_alpha = np.cos(np.deg2rad(alpha))
    normal_alignment = np.einsum('ij,ij->i', normals, surface_normals)
    normal_compatible = normal_alignment >= cos_alpha

    # Points are compatible if both the distance and normal conditions are met
    compatible = distance_compatible & normal_compatible

    return compatible.astype(float)  # Return a score of 1 for compatible points, 0 otherwise

def ransac_cone(points, normals, threshold, iterations, epsilon, alpha, min_height=None, max_height=None):
    best_inliers = []
    best_params = None
    best_score = 0

    # Divide points into top (for apex) and base (for base points)
    split_height = np.percentile(points[:, 2], 90)  # Top 10% for apex sampling
    top_points = points[points[:, 2] > split_height]
    base_points = points[points[:, 2] <= split_height]

    for i in range(iterations):
        # Sample apex from top points and base points from the bottom 90%
        apex_sample = top_points[np.random.choice(top_points.shape[0], 1, replace=False)]
        base_sample = base_points[np.random.choice(base_points.shape[0], 4, replace=False)]

        sample = np.vstack([apex_sample, base_sample])

        # Fit cone using the sampled apex and base points
        params = fit_cone(sample)
        if params is None:
            continue

        x0, y0, z0, a, b, c, theta, height = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)

        inliers = []
        inlier_indices = []
        for idx, point in enumerate(points):
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            distance = np.abs(perpendicular_distance - expected_distance)

            # Check if the point is an inlier and within the finite height of the cone
            if distance < threshold and 0 <= projection_length <= height:
                inliers.append(point)
                inlier_indices.append(idx)

        inlier_points = np.array(inliers)
        if len(inlier_points) == 0:
            continue

        inlier_normals = normals[inlier_indices] if normals is not None else None
        scores = calculate_cone_score(inlier_points, inlier_normals, np.array([x0, y0, z0]), direction_vector, theta, epsilon, alpha)

        total_score = np.sum(scores)
        if total_score > best_score:
            best_inliers = inliers
            best_params = params
            best_score = total_score

    print(f"Best Score: {best_score}, Inliers Count: {len(best_inliers)}")
    return best_params, np.array(best_inliers)

In [129]:
import numpy as np
from scipy.optimize import least_squares

def fit_cone(points):
    def residuals(params, points):
        # Unpack cone parameters: apex (x0, y0, z0), direction (a, b, c), opening angle (theta), and height
        x0, y0, z0, a, b, c, theta, height = params
        apex = np.array([x0, y0, z0])
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector

        vector_to_point = points - apex
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(theta) * projection_length

        # Penalize points outside the height range of the cone
        below_apex = projection_length < 0
        above_base = projection_length > height
        residual = perpendicular_distance - expected_distance
        residual[below_apex | above_base] = np.inf  # Large penalty for points outside the height range

        residual = np.nan_to_num(residual, nan=1e6, posinf=1e6, neginf=1e6)
        return residual

    apex = points[0]  # First point is the apex
    base_points = points[1:5]  # Next four points define the base
    base_center = np.mean(base_points, axis=0)
    height_vector = base_center - apex
    height = np.linalg.norm(height_vector)
    initial_direction = height_vector / height
    initial_angle = np.arctan(np.mean(np.linalg.norm(base_points - base_center, axis=1)) / height)

    initial_guess = [*apex, *initial_direction, initial_angle, height]
    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def calculate_cone_score(points, normals, apex, axis_direction, opening_angle, epsilon, alpha):
    axis_direction /= np.linalg.norm(axis_direction)

    # Compute vectors from apex to each point
    vectors_to_points = points - apex
    projection_lengths = np.dot(vectors_to_points, axis_direction)
    projections = projection_lengths[:, np.newaxis] * axis_direction
    perpendicular_distance = np.linalg.norm(vectors_to_points - projections, axis=1)
    expected_distance = np.tan(opening_angle) * projection_lengths

    # Distance Condition: check if the point is within the epsilon-radius band
    distance_compatible = np.abs(perpendicular_distance - expected_distance) <= epsilon

    # Compute expected surface normals (unit vectors pointing outward from the cone's surface)
    surface_normals = vectors_to_points - projections
    surface_normals /= np.linalg.norm(surface_normals, axis=1)[:, np.newaxis]

    # Normal Condition: check if the normals deviate within the alpha angle threshold
    cos_alpha = np.cos(np.deg2rad(alpha))
    normal_alignment = np.einsum('ij,ij->i', normals, surface_normals)
    normal_compatible = normal_alignment >= cos_alpha

    # Points are compatible if both the distance and normal conditions are met
    compatible = distance_compatible & normal_compatible

    return compatible.astype(float)  # Return a score of 1 for compatible points, 0 otherwise

def ransac_cone(points, normals, threshold, iterations, epsilon, alpha, min_height=None, max_height=None):
    best_inliers = []
    best_params = None
    best_score = 0

    # Divide points into top (for apex) and base (for base points)
    split_height = np.percentile(points[:, 2], 90)  # Top 10% for apex sampling
    top_points = points[points[:, 2] > split_height]
    base_points = points[points[:, 2] <= split_height]

    for i in range(iterations):
        # Sample apex from top points and base points from the bottom 90%
        apex_sample = top_points[np.random.choice(top_points.shape[0], 1, replace=False)]
        base_sample = base_points[np.random.choice(base_points.shape[0], 4, replace=False)]

        sample = np.vstack([apex_sample, base_sample])

        # Fit cone using the sampled apex and base points
        params = fit_cone(sample)
        if params is None:
            continue

        x0, y0, z0, a, b, c, theta, height = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)

        inliers = []
        inlier_indices = []
        for idx, point in enumerate(points):
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            distance = np.abs(perpendicular_distance - expected_distance)

            # Check if the point is an inlier and within the finite height of the cone
            if distance < threshold and 0 <= projection_length <= height:
                inliers.append(point)
                inlier_indices.append(idx)

        inlier_points = np.array(inliers)
        if len(inlier_points) == 0:
            continue

        inlier_normals = normals[inlier_indices] if normals is not None else None
        scores = calculate_cone_score(inlier_points, inlier_normals, np.array([x0, y0, z0]), direction_vector, theta, epsilon, alpha)

        total_score = np.sum(scores)
        if total_score > best_score:
            best_inliers = inliers
            best_params = params
            best_score = total_score

    print(f"Best Score: {best_score}, Inliers Count: {len(best_inliers)}")
    return best_params, np.array(best_inliers)

In [132]:
# Adjust the normal compatibility to reduce cylindrical inliers
def calculate_cone_score(points, normals, apex, axis_direction, opening_angle, epsilon, alpha):
    axis_direction /= np.linalg.norm(axis_direction)
    
    # Compute vectors from apex to each point
    vectors_to_points = points - apex
    projection_lengths = np.dot(vectors_to_points, axis_direction)
    projections = projection_lengths[:, np.newaxis] * axis_direction
    perpendicular_distance = np.linalg.norm(vectors_to_points - projections, axis=1)
    expected_distance = np.tan(opening_angle) * projection_lengths
    
    # Distance Condition: check if the point is within the epsilon-radius band
    distance_compatible = np.abs(perpendicular_distance - expected_distance) <= epsilon
    
    # Compute expected surface normals (unit vectors pointing outward from the cone's surface)
    surface_normals = vectors_to_points - projections
    surface_normals /= np.linalg.norm(surface_normals, axis=1)[:, np.newaxis]
    
    # Penalize if the normals are too perpendicular to the axis (suggesting a cylinder)
    cone_normal_deviation = np.abs(np.dot(surface_normals, axis_direction))
    
    # Normal Condition: check if the normals deviate within the alpha angle threshold
    cos_alpha = np.cos(np.deg2rad(alpha))
    normal_alignment = np.einsum('ij,ij->i', normals, surface_normals)
    normal_compatible = normal_alignment >= cos_alpha
    
    # Penalize points with normal deviations that are too cylindrical
    normal_compatible &= cone_normal_deviation < 0.9  # Adjust threshold to control cylinder detection
    
    # Points are compatible if both the distance and normal conditions are met
    compatible = distance_compatible & normal_compatible
    
    return compatible.astype(float)  # Return a score of 1 for compatible points, 0 otherwise


## BASE

In [5]:
import numpy as np
from scipy.optimize import least_squares

def fit_cone(points):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta, height = params
        apex = np.array([x0, y0, z0])
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)

        vector_to_point = points - apex
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(theta) * projection_length

        below_apex = projection_length < 0
        above_base = projection_length > height
        residual = perpendicular_distance - expected_distance
        residual[below_apex | above_base] = np.inf

        residual = np.nan_to_num(residual, nan=1e6, posinf=1e6, neginf=1e6)
        return residual

    apex = points[0]
    base_points = points[1:5]
    base_center = np.mean(base_points, axis=0)
    height_vector = base_center - apex
    height = np.linalg.norm(height_vector)
    initial_direction = height_vector / height
    initial_angle = np.arctan(np.mean(np.linalg.norm(base_points - base_center, axis=1)) / height)

    initial_guess = [*apex, *initial_direction, initial_angle, height]
    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def ransac_cone(points, normals, threshold, iterations, epsilon, alpha, min_opening_angle=np.deg2rad(5)):
    best_inliers = []
    best_params = None
    best_score = 0

    split_height = np.percentile(points[:, 2], 90)
    top_points = points[points[:, 2] > split_height]
    base_points = points[points[:, 2] <= split_height]

    for i in range(iterations):
        # Sample apex and base points
        apex_sample = top_points[np.random.choice(top_points.shape[0], 1, replace=False)]
        base_sample = base_points[np.random.choice(base_points.shape[0], 4, replace=False)]
        sample = np.vstack([apex_sample, base_sample])

        # Fit cone using the sampled apex and base points
        params = fit_cone(sample)
        if params is None:
            continue

        x0, y0, z0, a, b, c, theta, height = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)

        # Skip if the opening angle is too small (cylinder-like)
        if theta < min_opening_angle:
            continue

        # Inlier detection with penalty for non-increasing radius
        inliers = []
        score = 0  # Total score for the model
        prev_radius = -np.inf  # Set an initial value lower than any valid radius

        for idx, point in enumerate(points):
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)

            # Check for increasing radius
            if projection_length > 0:
                expected_distance = np.tan(theta) * projection_length
                distance = np.abs(perpendicular_distance - expected_distance)
                
                # Check if the point is an inlier
                if distance < threshold and projection_length <= height:
                    inliers.append(point)
                    
                    # Apply a penalty if the radius doesn't increase
                    if perpendicular_distance < prev_radius:
                        score -= 0.5  # Penalize points that don't follow radius growth
                    else:
                        score += 1  # Reward points that follow radius growth

                    prev_radius = perpendicular_distance  # Update previous radius for next check

        inlier_points = np.array(inliers)
        if len(inlier_points) == 0:
            continue

        # If this model has a better score than the current best, update the best model
        if score > best_score:
            best_inliers = inliers
            best_params = params
            best_score = score

    print(f"Best Score: {best_score}, Inliers Count: {len(best_inliers)}")
    return best_params, np.array(best_inliers)

In [6]:
best_params, best_inliers = ransac_cone(points, normals, threshold= 0.01, iterations= 1000, epsilon=0.01, alpha=15)

Best Score: 417.5, Inliers Count: 1637


In [7]:
cone_cloud = o3d.geometry.PointCloud()
cone_cloud.points = o3d.utility.Vector3dVector(best_inliers)
cone_cloud.paint_uniform_color([0,1,0])



#cone_mesh = create_cone_mesh(cone_params
#cone_mesh.paint_uniform_color([0.7, 1, 0.7])

surface_cloud.paint_uniform_color([0,0,0])

o3d.visualization.draw_geometries([surface_cloud, cone_cloud], point_show_normal=True)

In [392]:
import open3d as o3d
import numpy as np

# Function to create the cone mesh from the fitted cone parameters
def create_cone_mesh(cone_params, resolution=30):
    apex_x, apex_y, apex_z, dir_x, dir_y, dir_z, opening_angle, height = cone_params

    # Axis direction vector from the RANSAC model
    axis = np.array([dir_x, dir_y, dir_z], dtype=np.float64)  # Ensure float64 type
    axis /= np.linalg.norm(axis)  # Normalize the axis direction
    axis = -axis

    # Calculate the base radius using the opening angle and height
    base_radius = np.tan(opening_angle) * height

    # Create a cone mesh with Open3D's default parameters (apex at origin, base along positive z-axis)
    cone_mesh = o3d.geometry.TriangleMesh.create_cone(radius=base_radius, height=height, resolution=resolution)

    # Align the cone with the fitted axis direction
    default_axis = np.array([0, 0, 1])  # The default cone is along the z-axis in Open3D
    rotation_matrix = np.eye(4)  # Initialize a 4x4 identity matrix for the transformation

    # Compute the rotation to align the default axis with the RANSAC-fitted axis
    if not np.allclose(axis, default_axis):  # Check if the axes are already aligned
        axis_rotation = np.cross(default_axis, axis)  # Compute rotation axis using the cross product
        axis_rotation_norm = np.linalg.norm(axis_rotation)

        if axis_rotation_norm > 1e-8:  # Only apply rotation if needed
            axis_rotation /= axis_rotation_norm  # Normalize the rotation axis
            angle = np.arccos(np.dot(default_axis, axis))  # Compute the angle between the axes
            rotation_matrix[:3, :3] = o3d.geometry.get_rotation_matrix_from_axis_angle(axis_rotation * angle)

    # Apply the rotation to the cone mesh (to align it to the correct axis direction)
    cone_mesh.transform(rotation_matrix)

    # Translate the cone so that the apex is at the desired position (apex_x, apex_y, apex_z)
    translation_matrix = np.eye(4)  # Initialize a 4x4 identity matrix for translation
    translation_matrix[:3, 3] = [apex_x, apex_y, apex_z]  # Set the translation to move the apex
    cone_mesh.transform(translation_matrix)  # Apply the translation

    return cone_mesh

# Example usage:
# Assuming you have the cone parameters from the ransac_cone function:
# apex_x, apex_y, apex_z, dir_x, dir_y, dir_z, opening_angle, height
# Example params (use actual params from your model):

# Create cone mesh based on fitted cone parameters
cone_mesh = create_cone_mesh(best_params)

# Visualization code: display cone mesh along with point cloud and inliers
cone_cloud = o3d.geometry.PointCloud()
cone_cloud.points = o3d.utility.Vector3dVector(best_inliers)
cone_cloud.paint_uniform_color([0, 1, 0])

# Paint cone mesh with a uniform color
cone_mesh.paint_uniform_color([0.7, 1, 0.7])

# Paint the original point cloud black
downpcd.paint_uniform_color([0, 0, 0])

# Visualize the cone mesh along with the point cloud and inliers
o3d.visualization.draw_geometries([downpcd, cone_cloud, cone_mesh], point_show_normal=True)

In [404]:
import numpy as np
from scipy.optimize import least_squares

def fit_cone(points):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta, height = params
        apex = np.array([x0, y0, z0])
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)

        vector_to_point = points - apex
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(theta) * projection_length

        below_apex = projection_length < 0
        above_base = projection_length > height
        residual = perpendicular_distance - expected_distance
        residual[below_apex | above_base] = np.inf

        residual = np.nan_to_num(residual, nan=1e6, posinf=1e6, neginf=1e6)
        return residual

    apex = points[0]
    base_points = points[1:5]
    base_center = np.mean(base_points, axis=0)
    height_vector = base_center - apex
    height = np.linalg.norm(height_vector)
    initial_direction = height_vector / height
    initial_angle = np.arctan(np.mean(np.linalg.norm(base_points - base_center, axis=1)) / height)

    initial_guess = [*apex, *initial_direction, initial_angle, height]
    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def ransac_cone(points, normals, threshold, iterations, epsilon, alpha, min_opening_angle=np.deg2rad(30)):
    best_inliers = []
    best_params = None
    best_score = 0

    split_height = np.percentile(points[:, 2], 90)
    top_points = points[points[:, 2] > split_height]
    base_points = points[points[:, 2] <= split_height]

    for i in range(iterations):
        # Sample apex and base points
        apex_sample = top_points[np.random.choice(top_points.shape[0], 1, replace=False)]
        base_sample = base_points[np.random.choice(base_points.shape[0], 4, replace=False)]
        sample = np.vstack([apex_sample, base_sample])

        # Fit cone using the sampled apex and base points
        params = fit_cone(sample)
        if params is None:
            continue

        x0, y0, z0, a, b, c, theta, height = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)

        # Skip if the opening angle is too small (cylinder-like)
        if theta < min_opening_angle:
            continue

        # Inlier detection with penalty for non-increasing radius
        inliers = []
        score = 0  # Total score for the model
        prev_radius = -np.inf  # Set an initial value lower than any valid radius

        for idx, point in enumerate(points):
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)

            # Check for increasing radius
            if projection_length > 0:
                expected_distance = np.tan(theta) * projection_length
                distance = np.abs(perpendicular_distance - expected_distance)
                
                # Check if the point is an inlier
                if distance < threshold and projection_length <= height:
                    inliers.append(point)
                    
                    # Apply a penalty if the radius doesn't increase
                    if perpendicular_distance < prev_radius:
                        score -= 0.5  # Penalize points that don't follow radius growth
                    else:
                        score += 1  # Reward points that follow radius growth

                    prev_radius = perpendicular_distance  # Update previous radius for next check

        inlier_points = np.array(inliers)
        if len(inlier_points) == 0:
            continue

        # If this model has a better score than the current best, update the best model
        if score > best_score:
            best_inliers = inliers
            best_params = params
            best_score = score

    print(f"Best Score: {best_score}, Inliers Count: {len(best_inliers)}")
    return best_params, np.array(best_inliers)